In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install flask_ngrok
!pip install pyngrok==4.1.1
!pip install Werkzeug

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=3ab3a95dae28ffe42f280ca31e6a85f9e0053abebbb44ca62c9ee4ea5d9a8ece
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [3]:
import time

def run(instance_prompt,class_prompt,training_steps):

  HUGGINGFACE_TOKEN = ""

  #markdown Name/Path of the initial model.
  MODEL_NAME = "runwayml/stable-diffusion-v1-5"
  BRANCH = "fp16"


#markdown Enter instance prompt and class prompt.\
#markdown Example 1: photo of zwx person, photo of a person\
#markdown Example 2: photo of zwx toy, photo of a toy
  learning_rate = 1e-6 #param {type:"number"}
  output_file = "AI_PICS/models/my_dreambooth_model.safetensors" #param {type:"string"}
#markdown  Convert to fp16? (takes half the space (2GB)).
  fp16 = True #param {type: "boolean"}

#markdown Clear log after run?
  CLEAR_LOG = False #param {type:"boolean"}

# Mount google drive for saving the model
  from google.colab import drive
  drive.mount('/content/drive')

  OUTPUT_DIR = "/content/drive/MyDrive/output/output"
  INSTANCE_DIR = "/content/drive/MyDrive/data/instance"
  CLASS_DIR = "/content/drive/MyDrive/data/class"

  !mkdir -p $INSTANCE_DIR
  !mkdir -p $CLASS_DIR

  !rm -rf $OUTPUT_DIR
  !mkdir -p $OUTPUT_DIR

  if 'pipe' in locals():
    del pipe

# Check type of GPU and VRAM available.
  # !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
  !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# Upload training images.
  import os
  from google.colab import files
  import shutil


  time_start = time.time()
  def clear():
      from IPython.display import clear_output; return clear_output()

# huggingface token
  !mkdir -p ~/.huggingface
  !echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token


# install diffusers
  !git clone https://github.com/sagiodev/diffusers.git #https://github.com/huggingface/diffusers.git
  %cd diffusers
  !git checkout 08b453e3828f80027d881bb460716af95e192bcd -- ./scripts/convert_diffusers_to_original_stable_diffusion.py
  %pip install .


# install dreambooth
  %cd /content/diffusers/examples/dreambooth
  %pip install -r requirements.txt
  %pip install bitsandbytes #xformers torch==2.1.0+cu121 # gradio natsort safetensors xformers torch==2.1.0+cu121 accelerate
  !accelerate config default


# test bitsandbytes
  import bitsandbytes
  import torch
  print(torch.__version__)

# Dreambooth training - Edit this section to customize parameters
  !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --revision=$BRANCH \
    --instance_prompt="$instance_prompt" \
    --class_prompt="$class_prompt" \
    --class_data_dir=$CLASS_DIR \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --with_prior_preservation --prior_loss_weight=1.0 \
    --seed=1337 \
    --resolution=512 \
    --train_batch_size=1 \
    --train_text_encoder \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=$learning_rate \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --num_class_images=50 \
    --sample_batch_size=4 \
    --max_train_steps=$training_steps

#  --enable_xformers_memory_efficient_attention \

########################################
#prompt = "photo of zwx"
  prompt = instance_prompt
  negative_prompt = ""
  num_samples = 4
  guidance_scale = 7.5
  num_inference_steps = 25
  height = 512
  width = 512
  seed = 100

  %cd /content/
  from diffusers import StableDiffusionPipeline,EulerDiscreteScheduler
  from matplotlib.pyplot import figure, imshow, axis
  from matplotlib.image import imread
  import numpy as np



  if 'pipe' not in locals():
    pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    g_cuda = None




  g_cuda = torch.Generator(device='cuda')

  g_cuda.manual_seed(seed)

  from torch import autocast
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images

      fig=figure(figsize=(5*num_samples,5*num_samples))
      for i in range(num_samples):
          a=fig.add_subplot(1, num_samples,i+1)
          imshow(images[i])
          axis('off')


######################################
# Save to checkpoint file
  ckpt_path = '/content/drive/MyDrive/Data_File' + output_file
# save model file. Save to a new name if already exist.
  dirname= os.path.dirname(ckpt_path)
  !mkdir -p $dirname
  filename = os.path.basename(ckpt_path)
  fileanmeWithoutExt = os.path.splitext(filename)[0]
  ExtName = os.path.splitext(filename)[1]
  filenamePattern = fileanmeWithoutExt + '%d' + ExtName
  i = 1
  while os.path.isfile(ckpt_path):
    filename = filenamePattern%i
    ckpt_path = dirname + '/' + filename
    i += 1

  half_arg = "--half" if fp16 else ""
  !python /content/diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py \
      --use_safetensors \
      --model_path $OUTPUT_DIR \
      --checkpoint_path $ckpt_path $half_arg
  print(f"[*] Converted ckpt saved at {ckpt_path}")
  minutes = (time.time()-time_start)/60
  print("Dreambooth completed successfully. It took %1.1f minutes."%minutes)

In [4]:
#@title Test image generation from model
def run2(prompt,negative_prompt,num_samples ,
  guidance_scale ,
  height ,
  width ):
  # install diffusers
  !git clone https://github.com/sagiodev/diffusers.git #https://github.com/huggingface/diffusers.git
  %cd diffusers
  !git checkout 08b453e3828f80027d881bb460716af95e192bcd -- ./scripts/convert_diffusers_to_original_stable_diffusion.py
  %pip install .


# install dreambooth
  %cd /content/diffusers/examples/dreambooth
  %pip install -r requirements.txt
  %pip install bitsandbytes #xformers torch==2.1.0+cu121 # gradio natsort safetensors xformers torch==2.1.0+cu121 accelerate
  !accelerate config default

  import bitsandbytes
  import torch
  seed = -1
  num_inference_steps = 50

  %cd /content/
  from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
  from matplotlib.pyplot import figure, imshow, axis
  from matplotlib.image import imread
  import numpy as np
  OUTPUT_DIR = "/content/drive/MyDrive/output/output"
  INSTANCE_DIR = "/content/drive/MyDrive/data/instance"
  CLASS_DIR = "/content/drive/MyDrive/data/class"
  if 'pipe' not in locals():
    pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    g_cuda = None



  g_cuda = torch.Generator(device='cuda')

  g_cuda.manual_seed(seed)

  from torch import autocast
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images
      output_directory = "/content/drive/MyDrive/Colab Notebooks/FYP/static"
      i=1
      for im in images:
        filename = "generated_image_"+str(i)+".png"
        print(filename)
        filepath = os.path.join(output_directory, filename)
        # Save the image
        im.save(filepath)
        i=i+1
        print()

    # Optionally, display the saved filepath
      print(f"Image saved to: {filepath}")
      from ipywidgets import widgets, HBox
      from IPython.display import display
      for im in images:
          display(im)


In [ ]:
!ngrok authtoken 2dEysJgwtjumyUuhWZxkmMwuCU6_2nEaorudYLXqoRzYHonjt


from google.colab import files
from flask import Flask, render_template, request, redirect, url_for,g , jsonify
import sqlite3
from flask_ngrok import run_with_ngrok as r
from google.colab import files
import shutil
from werkzeug.utils import secure_filename
import os
import threading
import numpy as np
from PIL import Image
from pyngrok import ngrok
import base64
from io import BytesIO
from zipfile import ZipFile
# from google.colab import drive
# drive.mount('/content/drive')

page1="yes"
db_path = '/content/drive/MyDrive/Colab Notebooks/FYP/imagination.db'
conn = sqlite3.connect(db_path, check_same_thread=False)
cursor = conn.cursor()
# Check if the table already exists

conn.execute('''
    CREATE TABLE IF NOT EXISTS `Prompts For Model Training` (
        `Instance Prompt` STRING NOT NULL,
        `Class Prompt` STRING NOT NULL,
        `Training Steps` INTEGER NOT NULL
    );
''')
cursor.execute('''
    CREATE TABLE IF NOT EXISTS `Image Generation Prompts` (
       `Prompt` STRING NOT NULL,
        `Negative Prompt` STRING ,
        `Sample Size` INTEGER NOT NULL,
        `Guidance Scale` FLOAT NOT NULL
    );
''')
conn.commit()

def get_db():
    if 'db' not in g:
        g.db = conn
    return g.db
# Function to close the SQLite connection
def close_db(e=None):
    pass  # Do nothing, as the connection is shared
# Register close_db function to be called when the app context ends
# app.teardown_appcontext(close_db)




# Specify the path of the folder you want to create
folder_path = '/content/drive/MyDrive/data/instance'

# Check if the folder already exists
if not os.path.exists(folder_path):
    # If not, create the folder
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

templates_folder = '/content/drive/MyDrive/Colab Notebooks/FYP/templates'
static_folder = '/content/drive/MyDrive/Colab Notebooks/FYP/static'
app = Flask(__name__,template_folder = templates_folder,static_folder = static_folder)







status1 = {'status': 'in_progress'}



def process_request1(iprompt,cprompt,steps):
    with app.app_context():
        try:
            global page1
            run(iprompt,cprompt,steps)
            page1="yes"
            status1['status'] = 'completed'
            return jsonify(status1)
        except Exception as e:
            status1['status'] = 'error'
            error_message = f"An error occurred: {str(e)}"
            return jsonify({'status': 'error', 'message': error_message})
i=0
@app.route('/check_status', methods=['POST'])
def check_status1():
    global i
    print(1)
    if i==0:
        L=add_numbers()
        iprompt=L[0]
        cprompt=L[1]
        steps=L[2]
        i+=1
        processing_thread = threading.Thread(target=process_request1,args=(iprompt,cprompt,steps))
        processing_thread.start()
    return jsonify(status1)



@app.route('/process1')
def process1():
  return render_template('result.html')



###################################################


@app.route('/')
def index():
  return render_template('index.html',page1=page1)

def add_numbers():
    try:
          !ngrok authtoken 2dEysJgwtjumyUuhWZxkmMwuCU6_2nEaorudYLXqoRzYHonjt
          folder_path='/content/drive/MyDrive/data/instance'
          files = os.listdir(folder_path)

    # Iterate over the files and delete each one
          for file in files:
            file_path = os.path.join(folder_path, file)
            try:
              os.remove(file_path)
              print(f"File '{file_path}' deleted successfully.")
            except Exception as e:
              print(f"Error deleting file '{file_path}': {e}")
          INSTANCE_DIR = "/content/drive/MyDrive/data/instance"
          iprompt = str(request.form.get('data1'))
          print('done')
          cprompt = str(request.form.get('data2'))
          print('done')
          steps = int(request.form.get('data3'))
          print('done')
          n=int(request.form.get('length_i'))
          for i in range(n):
            file0= request.files.getlist('file'+str(i))
            for x in file0:
              filename=secure_filename(x.filename)
              x.save(os.path.join(INSTANCE_DIR,filename))
          print('done')

          cursor.execute("INSERT INTO `Prompts For Model Training` (`Instance Prompt`, `Class Prompt`, `Training Steps`) VALUES ( ?, ?, ? )", (iprompt, cprompt, steps ))
          conn.commit()
          return [iprompt,cprompt,steps]

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return render_template('error.html', message=error_message)


@app.route('/add0', methods=['POST'])
def two_page():
  try :
    !ngrok authtoken 2dEysJgwtjumyUuhWZxkmMwuCU6_2nEaorudYLXqoRzYHonjt
    prompt = str(request.form['prompt'])
    nprompt = str(request.form['nprompt'])
    sample = int(request.form['sample'])
    gs = float(request.form['gs'])
    height = int(request.form['height'])
    width = int(request.form['width'])
    cursor.execute("INSERT INTO `Image Generation Prompts`('Prompt', `Negative Prompt`, `Sample Size`, `Guidance Scale`) VALUES ( ?, ?, ?, ? )", (prompt, nprompt, sample, gs ))
    conn.commit()
    run2(prompt,nprompt,sample,gs,height,width)
    image_filenames=[]
    for x in range(1,sample+1):
      image_filenames.append("generated_image_"+str(x)+".png")
    return render_template('final.html', image_filenames=image_filenames)
  except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return render_template('error.html', message=error_message)



@app.route('/aboutus')
def aboutus():
    return render_template('aboutus.html')

r(app)
app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Folder '/content/drive/MyDrive/data/instance' already exists.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
import sqlite3

# Replace 'your_database.db' with the actual database file path
db_path = '/content/drive/MyDrive/Colab Notebooks/FYP/imagination.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute a SELECT query to retrieve all rows from the 'users' table
cursor.execute('SELECT * FROM `Prompts For Model Training`')

# Fetch all rows and print the results
rows = cursor.fetchall()

for row in rows:
    print(row)

# # Close the connection
# conn.close()

In [ ]:
import sqlite3

# Replace 'your_database.db' with the actual database file path
db_path = '/content/drive/MyDrive/Colab Notebooks/FYP/imagination.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute a SELECT query to retrieve all rows from the 'users' table
cursor.execute('SELECT * FROM `Image Generation Prompts`')

# Fetch all rows and print the results
rows = cursor.fetchall()

for row in rows:
    print(row)

# # Close the connection
# conn.close()
